In [27]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [28]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-20 15:24:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.79MB/s    in 0.2s    

2021-06-20 15:24:24 (5.79 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [35]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [36]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [37]:
# Step 1
df1 = vine_df.filter('total_votes>=20')
df1.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R3EM8C9CKA6GPK|          2|           41|        115|   N|                N|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|     

In [67]:
# Step 2.1
math = df1.withColumn('ratio',df1['helpful_votes'] / df1['total_votes'])
math.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|              0.84|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|               1.0|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|0.9736842105263158|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|         0.6640625|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|0.5697674418604651|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|0.7058823529411765|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y| 0.948051948051948|
|R3EM8C9CKA6GPK|          2|  

In [64]:
# Step 2.2
df2 = math.filter("ratio>=0.5")
df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|              0.84|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|               1.0|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|0.9736842105263158|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|         0.6640625|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|0.5697674418604651|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|0.7058823529411765|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y| 0.948051948051948|
|R29THVJFO35FZA|          5|  

In [125]:
# Step 3.1 - count of Vine reviews
df3 = df2.filter('vine="Y"').show()
df3 = df2.filter('vine="Y"')

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|               0.8|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N| 0.926829268292683|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|0.9361702127659575|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N| 0.967741935483871|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|0.9136690647482014|
| RGXLFTR31J561|          5|           29|         34|   Y|                N|0.8529411764705882|
|R2ZS3WCJYUDQH0|          3|          101|        115|   Y|                N|0.8782608695652174|
|R31771VBPRDVJS|          4|  

In [134]:
# Step 3.2 - count of Vine reviews
r_vine = df3.count()
r_vine

607

In [128]:
# Step 3.3 - 5 star vine reviews
fivestarpaid = df3.filter('star_rating=5').count()
fivestarpaid

257

In [130]:
# Step 4.1 - count of non Vine reviews
df4 = df2.filter('vine="N"').show()
df4 = df2.filter('vine="N"')

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|              0.84|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|               1.0|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|0.9736842105263158|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|         0.6640625|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|0.5697674418604651|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|0.7058823529411765|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y| 0.948051948051948|
|R29THVJFO35FZA|          5|  

In [135]:
# Step 4.2 - count of non Vine reviews
non_vine = df4.count()
non_vine

50522

In [132]:
# Step 4.3 - 5 star non vine reviews
fivestarunpaid = df4.filter('star_rating=5').count()
fivestarunpaid

25220

In [108]:
# Step 5.1 - total number of reviews
total = df2.select('review_id').count()
total

51129

In [87]:
# Step 5.2 - number of 5 star reviews
fivestar = df2.filter('star_rating=5').count()
fivestar

25477

In [136]:
# Step 5.3 - number of 5 star Vine reviews
p_paid = fivestarpaid / r_vine * 100
p_paid

42.33937397034596

In [137]:
# Step 5.4 - number of 5 star non Vine reviews
p_unpaid = fivestarunpaid / non_vine * 100
p_unpaid

49.91884723486798